### Jupyter notebook to run main using a parameter dummy

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from main import main
from utils import DefaultParams

In [3]:
params = DefaultParams(sinr=False)
#params.dataset.predictors = "loc"
params.local.sent_data_path = "/data/jdolli/glc23_data/SatelliteImages/"


"""params.model = "log_reg"
params.local.sent_data_path = "/data/jdolli/glc23_data/SatelliteImages/"
params.local.bioclim_lr_path = "/data/jdolli/glc23_data/sinr_data/data/env/bioclim_elevation_scaled_europe.npy"
params.local.dataset_file_path = "/data/jdolli/glc23_data/Pot_10_to_1000.csv"
params.local.cp_dir_path = "/scratch/jdolli/sent-sinr/checkpoints"
params.local.logs_dir_path = "/scratch/jdolli/sent-sinr/wandb_logs"
params.local.val_data_path = "/data/jdolli/glc23_data/Presence_Absence_surveys/loc_to_spec.csv"
"""

'params.model = "log_reg"\nparams.local.sent_data_path = "/data/jdolli/glc23_data/SatelliteImages/"\nparams.local.bioclim_lr_path = "/data/jdolli/glc23_data/sinr_data/data/env/bioclim_elevation_scaled_europe.npy"\nparams.local.dataset_file_path = "/data/jdolli/glc23_data/Pot_10_to_1000.csv"\nparams.local.cp_dir_path = "/scratch/jdolli/sent-sinr/checkpoints"\nparams.local.logs_dir_path = "/scratch/jdolli/sent-sinr/wandb_logs"\nparams.local.val_data_path = "/data/jdolli/glc23_data/Presence_Absence_surveys/loc_to_spec.csv"\n'

In [ ]:
main(params)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johannes-dollinger (ecovision). Use `wandb login --relogin` to force relogin


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | net  | SASI_MF | 10.7 M
---------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.997    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


Validation: 0it [00:00, ?it/s]

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
def load_cp(params, cp):
    if params.model == "sinr" or params.model == "log_reg":
        model = SINR.load_from_checkpoint(cp)
    elif "sat" in params.model:
        model = SAT_SINR.load_from_checkpoint(cp)